In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from calendar import monthrange
from pprint import pprint
from pandas.tseries.offsets import BDay

In [2]:
df = pd.read_csv("../../../Data/vw_Incident.csv",encoding='latin-1',low_memory=False)

In [3]:
from sklearn.utils import resample
df = resample(df, n_samples=10, random_state=1234)

In [4]:
def get_last_bdays_months():
    last_bdays = pd.date_range("2017.01.01", periods=11, freq='BM')
    last_bdays_offset = []
    for last_bday in last_bdays:
        last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
    return last_bdays_offset

def get_last_3_bdays_cutoffs(last_bdays):
    last_bdays_offset = []
    for last_bday in last_bdays:
        second_last = (last_bday - 2 * pd.DateOffset(BDay)).normalize() +pd.offset(hours=11)
        third_last = (last_bday - BDay).normalize() +pd.offset(hours=11)
        last_bdays_offset.append(third_last, second_last)
    return last_2_bdays

def days_of_the_month_cutoffs(df, last_bdays, last_3_bdays):
    # generate the daily cutoffs for every business day in the month
    start_month = df["Created_On"].min().month
    end_month = df["Created_On"].max().month
    print(end_month - start_month)
last_bdays_months = get_last_bdays_months()
last_3_bdays = get_last_3_bdays_cutoffs(last_bdays_months)
days_of_the_month_cutoffs(df, last_bdays_months, last_3_bdays)       

TypeError: int() argument must be a string, a bytes-like object or a number, not 'type'

In [ ]:
def get_end_of_day_seconds_left(date, last_bdays):
    last_33_hour_push = 33*60*60
    for i in range(len(last_bdays_offset)):    
        if date >= last_bdays_offset[i] and date < last_bdays_offset[i+1]:
            seconds_left = (last_bdays_offset[i+1] - date).seconds
            days_left = (last_bdays_offset[i+1] - date).days
            total_seconds_left = seconds_left + days_left*24*60*60  
            if total_seconds_left < last_33_hour_push:
                return total_seconds_left
            elif total_seconds_left < last_33_hour_push + 24 + 24:
                if date.day is sunday or saturday:
                    return total_seconds_left
                if date.day is Tue:
                    if date.time > 11pm:
                        return total_seconds_left
                    else:
                        return 11pm - date.time
                
                
            
            
        
        
        

In [ ]:
# def get_end_of_day_seconds_left(date, last_bdays_offset, last_2_bdays):
#     last_33_hour_push = 33*60*60
#     last_2_bdays = last_33_hour_push + (23+24+5)*60*60 
#     last_2_bdays_end_of_day = 11 *60*60
#     normal_end_of_day = 5 *60*60
#     time_of_day_in_seconds = date.hour*60*60 + date.minute*60 + date.second
#     print("time_of_day_in_seconds = ", time_of_day_in_seconds)
    
#     for i in range(len(last_bdays_offset)):       
# #         if date >= last_bdays_offset[i] and date <last_bdays_offset[i+1]:
            
#             seconds_left = (last_bdays_offset[i+1] - date).seconds
#             days_left = (last_bdays_offset[i+1] - date).days
#             total_seconds_left = seconds_left + days_left*24*60*60    
#             print("time left = ", last_bdays_offset[i+1] - date)
#             print("seconds_left = ", seconds_left)
#             print("days_left = ", days_left)
#             print("total_seconds_left = ", total_seconds_left)
#             # if it's during the last 33 hours (time between the last 23 hours cutoff and end of previous business day)
            
# #             if total_seconds_left <= last_33_hour_push:
# #                 print("last_33_hour_push")
# #                 return total_seconds_left
# #             elif date == last_bdays_offset[i+1] - BDay or date == last_bdays_offset[i+1] - BDay -BDay:
# #                 if date.time < 11pm:
# #                     return 11pm - date.time
# #                 else:
# #                     return 11pm + (12pm - date.time)
# #             else:
# #                 if date.time < 5pm:
# #                     return 5pm - date.time
# #                 else:
# #                     return 5pm + (12pm - date.time)
# #         elif date == BDay:
                
                
            
in_month = 4
in_date = 29
in_hour = 7
date = pd.datetime(2017, in_month, in_date, in_hour)

# last_bdays_months = get_last_bdays_months()
# get_end_of_day_seconds_left(date, last_bdays_months)